# Computation of layer by layer containment corrections




## Setup

In [2]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [3]:
# %load ../drawingTools


In [4]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v80'
version_V8 = version
version_V9 = 'v86'


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0', 'PU0', version_V8, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version_V8, 'V8')
    ]


samples_ele_V9 = [
    Sample('ele_flat2to100_PU0', 'PU0', version_V9, 'V9'),
    Sample('ele_flat2to100_PU200', 'PU200', version_V9, 'V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_photons_V8 = [
    Sample('photon_flat8to150_PU0', 'PU0', version_V8, 'V8'),
    Sample('photon_flat8to150_PU200', 'PU200', version_V8, 'V8')
    ]

samples_photons_V9 = [
    Sample('photon_flat8to150_PU0', 'PU0', version_V9, 'V9'),
    Sample('photon_flat8to150_PU200', 'PU200', version_V9, 'V9')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates_V8 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V8')
    ]

samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V9')
    ]



gen_part_selections: 8


Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU0_v86.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU200_v86.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_v86.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_pion_flat2to100_PU0_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu0_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v80.root does not exist


In [5]:
# %load settings.py
# %load settings.py
# from samples import get_label_dict
import python.selections as selections
import python.collections as collections

import pprint



def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary



# === samples =====================================================
samples = []

samples = samples_photons_V9
# samples += samples_ele_V9


sample = 'V9'

do_rate = False

# === TP ==========================================================
# tps = ['EG', 'TkEG', 'TkEle', 'TkIsoEle']
tps = ['HMvDR', 'HMvDRcylind10', 'HMvDRcylind5', 'HMvDRcylind2p5']

# === TP selections ================================================
tp_select = {}
tp_select['DEF'] = ['Em', 'all']
tp_select['DEFNC'] = tp_select['DEF']
tp_select['HMvDR'] = tp_select['DEF']
tp_select['HMvDRNC0'] = tp_select['DEF']
tp_select['HMvDRNC1'] = tp_select['DEF']
tp_select['HMvDRRebin'] = tp_select['DEF']
tp_select['HMvDRsTC'] = tp_select['DEF']
tp_select['HMvDRTrue'] = tp_select['DEF']
tp_select['HMvDRcylind10'] = tp_select['DEF']
tp_select['HMvDRcylind5'] = tp_select['DEF']
tp_select['HMvDRcylind2p5'] = tp_select['DEF']


tp_select['EG'] = ['EGq2', 'EGq3']

tp_select['TkEG'] = get_label_dict(selections.tkeg_qual_selections).keys()
tp_select['TkEle'] = ['EGq2', 'EGq3']

tp_select['TkEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['TkIsoEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['L1Trk'] = ['all', 'Pt2', 'Pt10']

# ==== GEN selections ===============================================

gen_select = ['GENEtaBC', 'GENEtaD']
# gen_select = ['GEN', 'GENEtaA', 'GENEtaB', 'GENEtaC', 'GENEtaD', 'GENEtaE',
#               'GENEtaAB', 'GENEtaABC', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE' ]

# tp_select['TkEG'] = ['EGq2EtaBC', 'EGq3EtaBC', 'EGq2EtaBCM2', 'EGq3EtaBCM2', 'EGq2EtaBCM2s', 'EGq3EtaBCM2s','EGq2EtaBCM3', 'EGq3EtaBCM3', 'EGq2EtaBCM3s', 'EGq3EtaBCM3s','EGq2EtaBCM4', 'EGq3EtaBCM4']



event_manager = collections.EventManager()

all_tpsets = {}
for collection in event_manager.collections:
    all_tpsets[collection.name] = collection.label


# all_tpsets = {'DEF': 'dRC3d',
#               'DEFCalib': 'NNDR Calib v1',
#               'DEFNC': 'dRC3d + new Th',
#               'HMvDR': 'HistoMaxC3d + dR(layer)',
#               'HMvDRNC0': 'HMC3d+dR(layer)+NC0',
#               'HMvDRNC1': 'HMC3d+dR(layer)+NC1',
#               'EG': 'EG',
#               'TkEG': 'TkEG',
#               'TkEle': 'TkEle',
#               'TkIsoEle': 'TkIsoEle',
#               'L1Trk': 'L1Track'}


# ==== adapt the plot retrieval
tpsets = {}
tpset_selections = {}
gen_selections = {}

for tp in tps:
    tpsets[tp] = all_tpsets[tp]

if 'DEF' in tps or 'HMvDR' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tp_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tp_match_selections))
if 'EG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.eg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.eg_pt_selections))
if 'TkEG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkeg_pt_selections))
if 'TkEle' in tps or 'TkIsoEle' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkisoeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkisoeg_pt_selections))
if 'L1Trk' in tps:
    tpset_selections.update(get_label_dict(selections.tracks_selections))


gen_selections.update(get_label_dict(selections.gen_part_selections))
gen_selections.update({'nomatch': ''})


pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
print gen_select


[EventManager] registering collection: MC
[EventManager] registering collection: GEN
[EventManager] registering collection: TC
[EventManager] registering collection: TCTrue
[EventManager] registering collection: DEF2D
[EventManager] registering collection: DEF2DTrue
[EventManager] registering collection: HMvDRTrue
[EventManager] registering collection: DEF
[EventManager] registering collection: DEFNC
[EventManager] registering collection: HMvDR
[EventManager] registering collection: HMvDRRebin
[EventManager] registering collection: HMvDRsTC
[EventManager] registering collection: HMvDRNC0
[EventManager] registering collection: HMvDRNC1
[EventManager] registering collection: DEFMerged
[EventManager] registering collection: DEFCalib
[EventManager] registering collection: HMvDRMerged
[EventManager] registering collection: HMvDRfixed
[EventManager] registering collection: HMvDRcylind10
[EventManager] registering collection: HMvDRcylind5
[EventManager] registering collection: HMvDRcylind2p5


In [6]:
tpsets


{'HMvDR': 'HM+dR(layer) Cl3d',
 'HMvDRcylind10': 'HM Cylinder 10cm',
 'HMvDRcylind2p5': 'HM Cylinder 2.5cm',
 'HMvDRcylind5': 'HM Cylinder 5cm'}

In [7]:
tpset_selections

{'Em': 'EGId',
 'EmPt10': 'EGId, p_{T}^{L1}>=10GeV',
 'EmPt20': 'EGId, p_{T}^{L1}>=20GeV',
 'EmPt30': 'EGId, p_{T}^{L1}>=30GeV',
 'Pt10': 'p_{T}^{L1}>=10GeV',
 'Pt20': 'p_{T}^{L1}>=20GeV',
 'Pt30': 'p_{T}^{L1}>=30GeV',
 'all': ''}

In [8]:
gen_selections

{'GEN': '',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'nomatch': ''}

In [9]:
from python.plotters import tp_calib_plotters

histo_primitives = pd.DataFrame() 

for plotter in tp_calib_plotters:
    histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)

    
print histo_primitives.data.unique()

['HMvDR' 'HMvDRcylind10' 'HMvDRcylind5' 'HMvDRcylind2p5']


In [10]:
files

{}

In [11]:
hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)

class HWrapper(object):
    def __init__(self, histo):
        self.histo = histo
    
    def get(self, debug=False):
        return self.histo


## Computation of the values

In [12]:
from sklearn.linear_model import LinearRegression
import array
import numpy as np 
for tp in tps:
    for gen_sel in gen_selections:
        if gen_sel == 'nomatch':
            continue
        for tp_sel in tp_select[tp]:
            print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)
            tree_name = '{}_{}_{}_calib'.format(tp, tp_sel, gen_sel)
            tree = samples[0].histo_file.Get(tree_name)
            layer_energy = rnp.tree2array(tree, branches=['e1', 'e3','e5','e7','e9','e11','e13','e15','e17','e19','e21','e23','e25','e27'], cache_size=400000000),
            target_energy = rnp.tree2array(tree, branches='Egen', cache_size=400000000)
#             print layer_energy
            layer_energy_list = [list(aset) for aset in layer_energy[0]]
            if len(layer_energy_list) == 0:
                continue
#             print layer_energy_list
            regression = LinearRegression(fit_intercept=False)
            regression.fit(np.array(layer_energy_list), np.array(target_energy))
            print 'Calibration Name: {} nclusters: {}, score: {}, coefficients:'.format(tree_name, len(layer_energy_list), regression.score(np.array(layer_energy_list), np.array(target_energy)))
            print [float("{0:0.2f}".format(i)) for i in regression.coef_ ]
            sumv = 0.
            for value in regression.coef_:
                sumv+=value
            print sumv
#                                       start=self._entry, stop=self._entry+1, cache_size=400000000)

            graph = ROOT.TGraph(len(regression.coef_), array.array('d', range(1,28,2)), array.array('d', regression.coef_))
            graph.SetMarkerStyle(7)
            graph.SetMarkerColor(2)
            stuff.append(graph)
            draw([graph], labels=['test'], text=tree_name, options='A', min_y=0., max_y=2.)
        
            hplot.data = hplot.data.append({'sample': samples[0].type,
                                           'pu': samples[0].label,
                                            'tp': tp,
                                            'tp_sel': tp_sel,
                                            'gen_sel': gen_sel,
                                            'classtype': 'pippo',
                                            'histo': HWrapper(graph),},
                                            ignore_index=True)

            

TP: HMvDR sel: Em, gen_sel: GENPt40
Calibration Name: HMvDR_Em_GENPt40_calib nclusters: 15644, score: 0.788226726465, coefficients:
[39.07, -1.28, 1.66, 0.11, 1.3, 1.2, 1.53, 0.87, 1.02, 0.47, 1.65, 0.91, -1.2, 7.69]
54.9954671934
TP: HMvDR sel: all, gen_sel: GENPt40
Calibration Name: HMvDR_all_GENPt40_calib nclusters: 17191, score: 0.765062562206, coefficients:
[43.73, -1.14, 0.94, 0.67, 0.47, 1.49, 2.2, 0.24, 1.5, 1.43, 1.09, -2.49, -6.59, 20.63]
64.17665039
TP: HMvDR sel: Em, gen_sel: GENEtaD
Calibration Name: HMvDR_Em_GENEtaD_calib nclusters: 4520, score: 0.846499156001, coefficients:
[-24.34, 1.15, 1.78, 0.33, 1.41, 1.34, 0.54, 1.05, 2.13, 0.56, 1.21, -1.85, 1.21, 6.34]
-7.15102639794
TP: HMvDR sel: all, gen_sel: GENEtaD
Calibration Name: HMvDR_all_GENEtaD_calib nclusters: 5447, score: 0.90314104625, coefficients:
[45.71, 4.77, 0.36, 1.02, 0.68, 1.55, 1.19, 0.77, 1.29, 1.33, 0.6, 0.16, 0.1, 5.09]
64.637611568
TP: HMvDR sel: Em, gen_sel: GENEtaBC
Calibration Name: HMvDR_Em_GENEtaBC

TP: HMvDRcylind5 sel: Em, gen_sel: GENEtaBC
Calibration Name: HMvDRcylind5_Em_GENEtaBC_calib nclusters: 10270, score: 0.990761539999, coefficients:
[4.91, 0.94, 1.09, 1.05, 0.92, 1.05, 1.11, 0.94, 1.02, 0.92, 1.1, 1.04, 1.34, 1.6]
19.0285899043
TP: HMvDRcylind5 sel: all, gen_sel: GENEtaBC
Calibration Name: HMvDRcylind5_all_GENEtaBC_calib nclusters: 12825, score: 0.992608759788, coefficients:
[6.59, 1.26, 0.89, 1.15, 0.97, 0.99, 1.05, 1.03, 1.06, 0.97, 0.86, 0.65, 0.9, 3.78]
22.1570113897
TP: HMvDRcylind5 sel: Em, gen_sel: GENPt30
Calibration Name: HMvDRcylind5_Em_GENPt30_calib nclusters: 16525, score: 0.80044405444, coefficients:
[40.41, -1.69, 1.97, 0.03, 1.42, 1.17, 1.41, 0.93, 1.21, 0.22, 1.68, 1.13, -1.01, 7.79]
56.6824969649
TP: HMvDRcylind5 sel: all, gen_sel: GENPt30
Calibration Name: HMvDRcylind5_all_GENPt30_calib nclusters: 18589, score: 0.778792611926, coefficients:
[44.56, -1.84, 1.62, 0.44, 0.69, 1.44, 1.93, 0.33, 1.6, 1.57, 1.29, -2.35, -7.09, 20.68]
64.8911517859
TP: HMvDR

In [107]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,V9,PU0,HMvDR,Em,GENPt40,pippo,<__main__.HWrapper object at 0x29da74110>
1,V9,PU0,HMvDR,all,GENPt40,pippo,<__main__.HWrapper object at 0x1093fbfd0>
2,V9,PU0,HMvDR,Em,GENEtaD,pippo,<__main__.HWrapper object at 0x29dc21590>
3,V9,PU0,HMvDR,all,GENEtaD,pippo,<__main__.HWrapper object at 0x29dab26d0>
4,V9,PU0,HMvDR,Em,GENEtaBC,pippo,<__main__.HWrapper object at 0x29dc05850>
5,V9,PU0,HMvDR,all,GENEtaBC,pippo,<__main__.HWrapper object at 0x11df49490>
6,V9,PU0,HMvDR,Em,GENPt30,pippo,<__main__.HWrapper object at 0x11de95890>
7,V9,PU0,HMvDR,all,GENPt30,pippo,<__main__.HWrapper object at 0x29dc104d0>
8,V9,PU0,HMvDR,Em,GENPt20,pippo,<__main__.HWrapper object at 0x11cc1dcd0>
9,V9,PU0,HMvDR,all,GENPt20,pippo,<__main__.HWrapper object at 0x29d776250>


In [79]:
hplot.data.tp.unique()

array(['HMvDR', 'HMvDRcylind10', 'HMvDRcylind5', 'HMvDRcylind2p5'], dtype=object)

In [80]:
hplot.labels_dict

{'Em': 'EGId',
 'EmPt10': 'EGId, p_{T}^{L1}>=10GeV',
 'EmPt20': 'EGId, p_{T}^{L1}>=20GeV',
 'EmPt30': 'EGId, p_{T}^{L1}>=30GeV',
 'GEN': '',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'HMvDR': 'HM+dR(layer) Cl3d',
 'HMvDRcylind10': 'HM Cylinder 10cm',
 'HMvDRcylind2p5': 'HM Cylinder 2.5cm',
 'HMvDRcylind5': 'HM Cylinder 5cm',
 'PU0': 'PU0',
 'PU200': 'PU200',
 'Pt10': 'p_{T}^{L1}>=10GeV',
 'Pt20': 'p_{T}^{L1}>=20GeV',
 'Pt30': 'p_{T}^{L1}>=30GeV',
 'V9': 'V9',
 'all': '',
 'nomatch': ''}

In [82]:
hfd = hplot.get_histo('pippo', 'V9', ['PU0'], ['HMvDRcylind2p5'], 'Em', 'GENEtaBC')

print hfd[0]
print hfd[1]
print hfd[2]




[<ROOT.TGraph object ("Graph") at 0x7f8ad0c5fcc0>]
['']
PU0, HM Cylinder 2.5cm, EGId, 1.52 < |#eta^{GEN}| <= 2.4


In [83]:
print hfd[2]


PU0, HM Cylinder 2.5cm, EGId, 1.52 < |#eta^{GEN}| <= 2.4


## Comparison of the calibration sets for different selections

In [108]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_selections:
            hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, tp_sel, gen_sel)
            draw(hsets, labels,  text=text, min_y=0., max_y=2.)

No match found for: pu: ['PU0'], tp: HMvDR, tp_sel: Em, gen_sel: nomatch, classtype: pippo


TypeError: object of type 'NoneType' has no len()

In [120]:

hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

<ROOT.TCanvas object ("c_97abe5") at 0x7f8ad720eba0>

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'all', 'GEN')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [125]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'all', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

<ROOT.TCanvas object ("c_96b9eb") at 0x7f8ad71849d0>

In [126]:
for tp in tps:
    hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['all', 'Em'], ['GENEtaBC'])
    draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind10'], ['all', 'Em'], ['GENEtaBC'])
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])